<h1>Multiview Registration RANSAC with P4KA</h1>

In [1]:
from registration_funcs import *
import AzureKinectTools, Frame, ObjectDetectionTools, keyboard, pyk4a, Tools
import time

VOXEL_SIZE = 30

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Read .mkv file from kinect

In [2]:
video = AzureKinectTools.AzureKinectTools('human_view_carrying_full_rotation.mkv')


video 1/1 (frame 1/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 78.8ms
video 1/1 (frame 2/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 10.0ms
video 1/1 (frame 3/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 13.0ms
video 1/1 (frame 4/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 12.0ms
video 1/1 (frame 5/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 10.0ms
video 1/1 (frame 6/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 11.0ms
video 1/1 (frame 7/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_full_rotation.mkv: 384x640 1 person, 1 chair, 1 tv, 11.0ms
video 1/1 (frame 8/1804) f:\\LIDAR\azurekinect\Tools\human_view_carrying_fu

Get point cloud A (source)

In [37]:
frame = video.get_frame(1) # get frame number n
pcd_points = frame.get_masked_point_cloud(1) # get masked point cloud for correct mask
pcd_colours = frame.get_point_cloud_colors(1) # get rgb data for correct mask
pcd_A = o3d.geometry.PointCloud() # create point cloud variable
pcd_A.points = o3d.utility.Vector3dVector(pcd_points) # add points
pcd_A.colors = o3d.utility.Vector3dVector(pcd_colours) # add colours
o3d.visualization.draw_geometries([pcd_A]) 

Crop point cloud A using DBSCAN

In [39]:
pcd_A_ds = pcd_A.voxel_down_sample(voxel_size=VOXEL_SIZE) # down sample first for speed
clusters, aabbs = get_clusters(pcd_A_ds) # get clusters and bounding boxes
pcd_A_crp = pcd_A.crop(aabbs[0]) # crop raw point cloud based on calculated clusters
o3d.visualization.draw_geometries([pcd_A_crp])

Get point cloud B (target)

In [25]:
frame = video.get_frame(100) # get frame number n
pcd_points = frame.get_masked_point_cloud(1) # get masked point cloud for correct mask
pcd_colours = frame.get_point_cloud_colors(1) # get rgb data for correct mask
pcd_B = o3d.geometry.PointCloud() # create point cloud variable
pcd_B.points = o3d.utility.Vector3dVector(pcd_points) # add points
pcd_B.colors = o3d.utility.Vector3dVector(pcd_colours) # add colours
o3d.visualization.draw_geometries([pcd_B]) 

Crop point cloud B using DBSCAN

In [40]:
pcd_B_ds = pcd_B.voxel_down_sample(voxel_size=VOXEL_SIZE) # down sample first for speed
clusters, aabbs = get_clusters(pcd_B_ds) # get clusters and bounding boxes
pcd_B_crp = pcd_B.crop(aabbs[1]) # crop raw point cloud based on correct cluster
o3d.visualization.draw_geometries([pcd_B_crp])

Downsample raw point clouds

In [41]:
pcd_A_ds = pcd_A_crp.voxel_down_sample(voxel_size=VOXEL_SIZE) # down sample cropped point clouds
pcd_B_ds = pcd_B_crp.voxel_down_sample(voxel_size=VOXEL_SIZE) 
print(f'Point cloud A has : {number_of_points(pcd_A_ds)} points')
print(f'Point cloud B has : {number_of_points(pcd_B_ds)} points')

Point cloud A has : 1336 points
Point cloud B has : 1278 points


Compute registration using down sampled data with down sampled output

In [42]:
start = time.perf_counter() # start timer
output_pcd = ransac_icp_registration(pcd_A_ds, pcd_B_ds, VOXEL_SIZE) # compute RANSAC registration for down sampled cropped point clouds
end = time.perf_counter() # stop timer
print(f'Registration time : {end-start}s') # print time
o3d.visualization.draw_geometries([output_pcd])
print(f'Final point cloud has : {number_of_points(output_pcd)} points')

For a voxel size of 30 : RegistrationResult with fitness=4.543413e-01, inlier_rmse=9.098621e+00, and correspondence_set size of 607
Access transformation to get result.
Registration time : 0.10707820000243373s
Final point cloud has : 2614 points


Compute registration using down sampled data with raw output

In [44]:
start = time.perf_counter() # start timer
transform = ransac_icp_registration_transform(pcd_A_ds, pcd_B_ds, VOXEL_SIZE) # compute RANSAC registration transformation on downsampled data for speed
pcd_A_crp.estimate_normals() # in order to apply transformation to raw data, the normals have to be calculated first
pcd_B_crp.estimate_normals() 
output_pcd = get_registration_result(pcd_A_crp, pcd_B_crp, transform) # apply transformation to raw data
end = time.perf_counter() # stop timer
print(f'Registration time : {end-start}s') # print time
o3d.visualization.draw_geometries([output_pcd])
print(f'Final point cloud has : {number_of_points(output_pcd)} points')

For a voxel size of 30 : RegistrationResult with fitness=4.558383e-01, inlier_rmse=9.101004e+00, and correspondence_set size of 609
Access transformation to get result.
Registration time : 0.6193001999927219s
Final point cloud has : 405712 points
